In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import pydicom
import cv2

# Define your data directory, batch size, image size, number of epochs, and number of classes
data_dir = r'D:\Alzheimers\Task1\Task1-Images\PET\Images'
batch_size = 32
image_size = (128, 128)
num_epochs = 20
num_classes = 5

def load_and_preprocess_images_and_labels(directory):
    # Function to load and preprocess images and labels
    image_data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".dcm"):
            dcm_path = os.path.join(directory, filename)
            try:
                dcm_data = pydicom.dcmread(dcm_path)
                img = dcm_data.pixel_array.astype(np.float32) / 255.0
                img = cv2.resize(img, image_size)
                image_data.append(img)

                filename_parts = filename.split('_')
                class_name = filename_parts[-1].split('.')[0]

                class_mapping = {'CN': 0, 'AD': 1, 'MCI': 2, 'EMCI': 3, 'LMCI': 4}
                label = class_mapping.get(class_name, -1)

                if label != -1:
                    labels.append(label)
                else:
                    print(f"Unknown class name: {class_name}")
            except Exception as e:
                print(f"Error processing file {dcm_path}: {str(e)}")

    return np.array(image_data), np.array(labels)

# Load and preprocess all data from the directory
all_images, all_labels = load_and_preprocess_images_and_labels(data_dir)

# Split the data into training and testing sets (80% training, 20% testing)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42
)

# Expand grayscale images to three channels (RGB)
train_images = np.repeat(train_images[:, :, :, np.newaxis], 3, axis=-1)
test_images = np.repeat(test_images[:, :, :, np.newaxis], 3, axis=-1)

# Convert labels to one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Calculate class weights to handle class imbalance
class_weights = class_weight.compute_sample_weight('balanced', train_labels.argmax(axis=1))
class_weights = dict(enumerate(class_weights))

# Create a data generator for training images with more augmentation
train_data_generator = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the EfficientNetB3 model
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))  # Use EfficientNetB3

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze some layers of the base model (you can experiment with the number of layers to freeze)
for layer in base_model.layers[:15]:
    layer.trainable = False

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the data generator
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    history = model.fit(
        train_data_generator.flow(train_images, train_labels, batch_size=batch_size),
        steps_per_epoch=len(train_images) // batch_size,
        epochs=1,
        class_weight=class_weights,  # Apply class weights
        verbose=1
    )

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Define the path where you want to save the model
save_path = r'D:\Alzheimers\Models\PET_Task1.h5'  

# Save the model to the specified location
model.save(save_path)


Epoch 1/20
1234/1234 [==============================] - 2628s 2s/step - loss: 0.8465 - accuracy: 0.4468
Epoch 2/20
1234/1234 [==============================] - 2596s 2s/step - loss: 0.7028 - accuracy: 0.5389
Epoch 3/20
1234/1234 [==============================] - 2597s 2s/step - loss: 0.5907 - accuracy: 0.6128
Epoch 4/20
1234/1234 [==============================] - 2597s 2s/step - loss: 0.4941 - accuracy: 0.6786
Epoch 5/20
1234/1234 [==============================] - 2600s 2s/step - loss: 0.4092 - accuracy: 0.7433
Epoch 6/20
1234/1234 [==============================] - 2596s 2s/step - loss: 0.3458 - accuracy: 0.7851
Epoch 7/20
1234/1234 [==============================] - 2588s 2s/step - loss: 0.2983 - accuracy: 0.8178
Epoch 8/20
1234/1234 [==============================] - 2599s 2s/step - loss: 0.2588 - accuracy: 0.8432
Epoch 9/20
1234/1234 [==============================] - 2610s 2s/step - loss: 0.2279 - accuracy: 0.8620
Epoch 10/20
1234/1234 [==============================] - 2608s 2

C:\Users\prana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import pydicom
import cv2

data_dir = r'D:\Alzheimers\Task1\Task1-Images\fMRI\Images'
batch_size = 8
image_size = (75, 75)
num_epochs = 5  
num_classes = 5  

def load_and_preprocess_images_and_labels(directory):
    image_data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".dcm"):
            dcm_path = os.path.join(directory, filename)
            try:
                dcm_data = pydicom.dcmread(dcm_path)
                img = dcm_data.pixel_array.astype(np.float32) / 255.0
                img = cv2.resize(img, image_size)
                image_data.append(img)
                
                filename_parts = filename.split('_')
                class_name = filename_parts[-1].split('.')[0]
                
                class_mapping = {'CN': 0, 'AD': 1, 'MCI': 2, 'EMCI': 3, 'LMCI': 4}
                label = class_mapping.get(class_name, -1) 
                
                if label != -1:
                    labels.append(label)
                else:
                    print(f"Unknown class name: {class_name}")
            except Exception as e:
                print(f"Error processing file {dcm_path}: {str(e)}")

    return np.array(image_data), np.array(labels)


# Load and preprocess all data from the directory
all_images, all_labels = load_and_preprocess_images_and_labels(data_dir)

# Split the data into training and testing sets (80% training, 20% testing)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42
)

# Expand grayscale images to three channels (RGB)
train_images = np.repeat(train_images[:, :, :, np.newaxis], 3, axis=-1)
test_images = np.repeat(test_images[:, :, :, np.newaxis], 3, axis=-1)

# Convert labels to one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Calculate class weights to handle class imbalance
class_weights = class_weight.compute_sample_weight('balanced', train_labels.argmax(axis=1))
class_weights = dict(enumerate(class_weights))

# Create a data generator for training images with more augmentation
train_data_generator = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(75, 75, 3))
# or, alternatively, use InceptionV3
# base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(96, 96, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout for regularization
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze some layers of the base model (you can experiment with the number of layers to freeze)
for layer in base_model.layers[:15]:
    layer.trainable = False

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the data generator
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    history = model.fit(
        train_data_generator.flow(train_images, train_labels, batch_size=batch_size),
        steps_per_epoch=len(train_images) // batch_size,
        epochs=1,
        class_weight=class_weights,  # Apply class weights
        verbose=1
    )

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Define the path where you want to save the model
save_path = r'D:\Alzheimers\Models\fMRI_Task1.h5'  # Replace with your desired path

# Save the model to the specified location
model.save(save_path)

Epoch 1/5
16019/16019 [==============================] - 2777s 173ms/step - loss: 0.5988 - accuracy: 0.7300
Epoch 2/5
16019/16019 [==============================] - 2845s 178ms/step - loss: 0.1452 - accuracy: 0.9457
Epoch 3/5
16019/16019 [==============================] - 2850s 178ms/step - loss: 0.0900 - accuracy: 0.9684
Epoch 4/5
16019/16019 [==============================] - 2836s 177ms/step - loss: 0.0680 - accuracy: 0.9770
Epoch 5/5
16019/16019 [==============================] - 2849s 178ms/step - loss: 0.0610 - accuracy: 0.9800
1002/1002 - 257s - loss: 0.0117 - accuracy: 0.9968 - 257s/epoch - 257ms/step
Test Accuracy: 99.68%


C:\Users\prana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
import os
import numpy as np
import tensorflow as tf
import pydicom
import cv2

# Load the saved model
model = tf.keras.models.load_model('D:/Alzheimers/Models/MRI_Task1.h5')  # Replace with the actual path to your saved model

# Define data directory and image size
data_dir = r'D:\Alzheimers\Task1\Task1-Images\MRI\Images'
image_size = (128, 128)  # Adjust this to match the input size your model expects

# Create a function to load and preprocess test images
def load_and_preprocess_test_images(directory):
    image_data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".dcm"):
            dcm_path = os.path.join(directory, filename)
            try:
                dcm_data = pydicom.dcmread(dcm_path)
                img = dcm_data.pixel_array.astype(np.float32) / 255.0
                img = cv2.resize(img, image_size)
                image_data.append(img)
                
                filename_parts = filename.split('_')
                class_name = filename_parts[-1].split('.')[0]
                
                class_mapping = {'CN': 0, 'AD': 1, 'MCI': 2, 'EMCI': 3, 'LMCI': 4}
                label = class_mapping.get(class_name, -1) 
                
                if label != -1:
                    labels.append(label)
                else:
                    print(f"Unknown class name: {class_name}")
            except Exception as e:
                print(f"Error processing file {dcm_path}: {str(e)}")

    return np.array(image_data), np.array(labels)

# Load and preprocess the test images
test_images, true_labels = load_and_preprocess_test_images(data_dir)

# Expand grayscale images to three channels (RGB)
test_images = np.repeat(test_images[:, :, :, np.newaxis], 3, axis=-1)

# Use the model to make predictions on the test data
predictions = model.predict(test_images)

# Obtain predicted labels (class indices with highest probability)
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy by comparing predicted and true labels
accuracy = np.mean(np.equal(predicted_labels, true_labels))

# Print the accuracy 
print("Test Accuracy:", accuracy * 100, "%")

528/528 [==============================] - 254s 475ms/step
Test Accuracy: 89.12270302311796 %
